In [1]:
import logging
import os.path
import six
import sys
import gensim
import warnings
from collections import Counter
import gensim as gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.word2vec import LineSentence
import jieba

import re

%matplotlib inline

# 训练好的词向量

In [2]:
###直接调用训练好的词向量  人民日报
from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('sgns.renmin.bigram-char') #sgns.renmin.txt'

In [5]:
wv2 = word_vectors.most_similar(u"日本", topn=20)  # 20个最相关的
print(u"和【日本】最相关的20个词：\n")
for word in wv2:
    print(word[0], word[1])

和【日本】最相关的20个词：

日本政府 0.6873617172241211
日本驻军 0.6509954929351807
韩国 0.6257189512252808
北海道 0.6161410808563232
德国 0.6079153418540955
冲绳 0.6024359464645386
日本国会 0.6020386815071106
石川县 0.5961585640907288
青森县 0.5940454006195068
日本关西地区 0.5918794274330139
福冈县 0.5896080732345581
日本帝国 0.5873615741729736
日本产 0.5869629979133606
日本自民党 0.5849270820617676
日本民主党 0.5845142006874084
西德 0.5832101106643677
日本松下 0.5827378630638123
日本大学 0.582249641418457
日本东京大学 0.5813157558441162
日本国 0.5788332223892212


In [10]:
print(word_vectors)
# 计算两个词的相似度/相关程度
try:
    y1 = word_vectors.similarity(u"北京大学", u"土木工程")
except KeyError:
    y1 = 0
print(u"【文学】和【数学】的相似度为：", y1)

KeyedVectors<vector_size=300, 356053 keys>
【文学】和【数学】的相似度为： 0.35078138


In [11]:
# 寻找不合群的词
y2 =word_vectors.doesnt_match(u"社会学 经济学 哲学 物理学".split())
print(u"不合群的词：", y2)


不合群的词： 物理学


# 百度API的方法

In [12]:
##安装
##pip install baidu-aip
##调用相关包
from aip import AipNlp

In [13]:
##常量APP_ID在百度云控制台中创建，常量API_KEY与SECRET_KEY是在创建完毕应用后，系统分配给用户的，均为字符串，用于标识用户，为访问做签名验证，可在AI服务控制台中的应用列表中查看。
##获取说明见链接：https://ai.baidu.com/ai-doc/REFERENCE/Ck3dwjgn3
APP_ID = '22937951'
API_KEY = 'sc***' #
SECRET_KEY = 'Gs***' #
client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [14]:
##词向量表示接口提供中文词向量的查询功能。
word="社会"
client.wordEmbedding(word)

{'log_id': 5784650443761269590,
 'word': '社会',
 'vec': [0.542227,
  0.864834,
  -0.414497,
  1.00286,
  0.195443,
  0.57845,
  -0.779943,
  1.75928,
  -0.446708,
  -0.320079,
  -1.72516,
  0.680588,
  -0.200344,
  0.450854,
  1.34213,
  -0.743724,
  0.818294,
  0.165789,
  -1.57067,
  0.84904,
  0.428487,
  1.01183,
  -0.825374,
  0.670236,
  -0.770021,
  -0.608155,
  -1.40447,
  -0.268275,
  -0.356322,
  -1.10026,
  1.29873,
  -0.00581356,
  -0.130957,
  -0.276089,
  -0.565437,
  0.423213,
  -1.12054,
  0.427069,
  -0.0585604,
  -0.29324,
  0.219869,
  -0.610123,
  0.43228,
  0.111803,
  0.367252,
  1.6539,
  0.0922434,
  -0.701276,
  1.47117,
  -0.198356,
  1.24673,
  -0.0273442,
  -0.745284,
  0.611476,
  1.66379,
  -0.29228,
  -0.0585908,
  -0.851333,
  0.138032,
  -0.331151,
  -0.260025,
  -0.389952,
  0.664605,
  0.460638,
  1.31709,
  -2.27218,
  0.13466,
  0.298847,
  -1.48316,
  0.241035,
  0.547518,
  -0.113135,
  -0.725092,
  0.754776,
  -1.33455,
  1.82498,
  -0.602632,
  -

In [15]:
##词义相似度：输入两个词，得到两个词的相似度结果。
result3 = client.wordSimEmbedding("社会","国家")

print(result3)

{'log_id': 1758127084179836342, 'score': 0.027637, 'words': {'word_2': '国家', 'word_1': '社会'}}


In [16]:
##短文本相似度接口用来判断两个文本的相似度得分。
result1 = client.simnet("这件衣服颜色怎么样？","这件衣服价格怎么样")

print(result1)

{'texts': {'text_1': '这件衣服颜色怎么样？', 'text_2': '这件衣服价格怎么样'}, 'score': 0.608503, 'log_id': 1738106241714888918}


文章分类
对文章按照内容类型进行自动分类，首批支持娱乐、体育、科技等26个主流内容类型，为文章聚类、文本内容分析等应用提供基础技术支持。

In [11]:
title = "欧洲冠军杯足球赛"

content = "欧洲冠军联赛是欧洲足球协会联盟主办的年度足球比赛，代表欧洲俱乐部足球最高荣誉和水平，被认为是全世界最高素质、最具影响力以及最高水平的俱乐部赛事，亦是世界上奖金最高的足球赛事和体育赛事之一。"

""" 调用文章分类 """
client.topic(title, content)

{'log_id': 2392256324346204822,
 'item': {'lv2_tag_list': [{'score': 0.915631, 'tag': '足球'},
   {'score': 0.803507, 'tag': '国际足球'},
   {'score': 0.77813, 'tag': '英超'}],
  'lv1_tag_list': [{'score': 0.830915, 'tag': '体育'}]}}

参数名称	类型	详细说明
item	object	分类结果，包含一级与二级分类
+lv1_tag_list	array of objects	一级分类结果
+lv2_tag_list	array of objects	二级分类结果
++score	float	类别标签对应得分，范围0-1
++tag	string	类别标签

In [34]:
##评论观点抽取接口用来提取一条评论句子的关注点和评论观点，并输出评论观点标签及评论观点极性。
text = "华为手机拍照很给力"

""" 调用评论观点抽取 """
client.commentTag(text);

""" 如果有可选参数 """
options = {}
options["type"] = 13

""" 带参数调用评论观点抽取 """
client.commentTag(text, options)

{'log_id': 922718605645685037,
 'items': [{'sentiment': 2,
   'abstract': '华为手机<span>拍照很给力</span>',
   'prop': '拍照',
   'begin_pos': 8,
   'end_pos': 18,
   'adj': '给力'}]}

# 论文

Garg, N., Schiebinger, L., Jurafsky, D., & Zou, J. (2018). Word embeddings quantify 100 years of gender and ethnic stereotypes. Proceedings of the National Academy of Sciences, 115(16), E3635-E3644.


Kozlowski, A. C., Taddy, M., & Evans, J. A. (2019). The geometry of culture: Analyzing the meanings of class through word embeddings. American Sociological Review, 84(5), 905-949.